In [40]:
from mesa import Model
from mesa.space import MultiGrid
from mesa import Agent
import random

In [51]:
class AmongUs(Model):
    def __init__(self, map, n_agents):
        self.height = 70
        self.width = 70
        
        self.grid = MultiGrid(self.width, self.height, torus=True)
        
        self.n_agents = n_agents
        self.agents = []

    def new_agent(self, agent_type, pos):
        '''
        Method that enables us to add agents of a given type.
        '''
        self.n_agents += 1
        
        # Create a new agent of the given type
        new_agent = agent_type(self.n_agents, self, pos)
        
        # Place the agent on the grid
        self.grid.place_agent(new_agent, pos)
        
        # And add the agent to the model so we can track it
        self.agents.append(new_agent)
        
    def remove_agent(self, agent):
        '''
        Method that enables us to remove passed agents.
        '''
        self.n_agents -= 1
        
        # Remove agent from grid
        self.grid.remove_agent(agent)
        
        # Remove agent from model
        self.agents.remove(agent)
        
    def step(self):
        '''
        Method that steps every agent. 
        
        Prevents applying step on new agents by creating a local list.
        '''
        for agent in list(self.agents):
            agent.step()
            agent.random_move()
            print(agent.pos)


    def run_model(self, step_count=10):
        '''
        Method that runs the model for a specific amount of steps.
        '''
        for i in range(step_count):
            self.step()


class RandomWalker(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos = pos

    def random_move(self):
        '''
        This method should get the neighbouring cells (Moore's neighbourhood), select one, and move the agent to this cell.
        '''
        grid = self.model.grid
        neighborhood = grid.get_neighborhood(self.pos, True)
        grid.move_agent(self, random.choice(neighborhood))
        
tester = AmongUs('map', 0)

# Create a RandomWalker, so that we can call the random_move() method
start_position = (5, 5)
tester.new_agent(RandomWalker, start_position)

# Create a reference, so that we can properly test
walker = tester.agents[0]

# Check movement
walker.random_move()

tester.run_model()

(5, 6)
(4, 5)
(5, 4)
(6, 5)
(7, 6)
(6, 5)
(5, 6)
(4, 7)
(4, 8)
(5, 8)
